[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alessandronascimento/BioMolComp/blob/main/P04/ConsultaBasesdeDados2.ipynb)

# Consulta em Bases de Dados - Parte 2 #

Nesta segunda seção, vamos retomar as consultas em bases de dados, mas desta vez usaremos o Biopython para esta finalidade.

Eventualmente o pesquisador possui muitas sequências e/ou entradas para pesquisar em uma base de dados com a Entrez, por exemplo. Nestes casos, executar a consulta das entradas uma a uma através do portal WWW pode ser um processo trabalhoso e demorado.

O NCBI oferece a possibilidade de consultas à base de dados a partir de programas ou scripts que acessam diretamente o banco de dados dos servidores do NCBI. As consultas realizadas desta forma têm algumas regras (limite de consultas por minuto, por exemplo) que têm que ser satisfeitas. Do contrário, o acesso pode ser bloqueado. 

Nesta seção da prática, faremos a consulta de algumas sequências do vírus SARS-Cov2 sem usar a interface web e empregando ferramentas do Biopython para esta finalidade. 

In [ ]:
#@title Preparando o Ambiente para a Execução

!pip3 install biopython

from Bio import SeqIO
from Bio.Seq import Seq
from Bio import Entrez
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

from google.colab import auth
auth.authenticate_user()

import requests

gcloud_token = !gcloud auth print-access-token
gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
#print(gcloud_tokeninfo['email'])
user_email = gcloud_tokeninfo['email']
print("User email %s will be used to autenticate Entrez access." % user_email)

### Consulta à Base de Dados Entrez Através do Biopython ###

Vamos consultar à base de dados Entrez. Esta base de dados **requer que o usuário se identifique ao fazer o login**. Por esta razão estamos usando a ferramenta de autenticação do Google na primeira célula de código. Esta ferramenta de autenticação permite que o email do usuário seja passado na variável user_email no código abaixo.

A autenticação é uma das regras de uso do Entrez. Caso um usuário exceda as regras de uso, ele *receberá um aviso por email antes de ser bloqueado pelo servidor*.

### Fazendo uma consulta à base de nucleotídeos

Nos códigos abaixo, vamos fazer uma consulta à base de dados de nucleotídeos (Genebank). O resultado vai ser mostrado na forma de texto simples com os resultados da entrada.

Vamos repetir a busca que fizemos na primeira parte desta prática: o genoma do vírus SARS-Cov2. A entrada deste genoma no banco de dados de nucleotídeos é o **NC_045512.2**.

Usando o formulário abaixo, coloque esta entrada no ID da base de dados de nucleotídeos e faça a busca na base de dados.



In [ ]:
nucleotide_id = "" #@param {type:"string"}

Entrez.email = user_email
handle = Entrez.efetch(db="nucleotide", id=nucleotide_id, rettype="gb", retmode="text")
print(handle.read())

### Fazendo uma consulta à base de nucleotídeos ###

Agora vamos repetir a busca que fizemos anteriormente, mas desta vez, restringiremos a região do genoma a uma região específica. Em particular, vamos restringir a nossa busca à região compreendida entre as bases 21560 e 25390. 

No formulário abaixo, defina esta região do genoma do vírus e coloque também (novamente) a entrada do genoma no banco de dados de nucleotídeos.

In [11]:
nucleotide_id = "" #@param {type:"string"}
seq_from =  #@param {type:"integer"}
seq_to =  #@param {type:"integer"}


Entrez.email = user_email
handle = Entrez.efetch(db="nucleotide", id=nucleotide_id, rettype="gb", retmode="text", seq_start=int(seq_from), seq_stop=int(seq_to))
record = SeqIO.read(handle, "genbank")
handle.close()


### Estendendo a Análise da Sequência ###

No código anterior, além de fazer a busca na base de dados, criamos uma variável *record*, que recebeu os dados que foram consultados. 

```
record = SeqIO.read(handle, "genbank")
```
Nesta variável podemos extrair características da consulta que fizemos.

Por exemplo, vamos listar o *ID da sequência*, a *descrição* e as *características*:


In [ ]:
print("Record Id: %s" % record.id)
print("Record name: %s" % record.name)
print("Record description: %s " % record.description)
print("Number of Features: %d" % len(record.features))

print("Record Features:")

for i in range(0,len(record.features)):
  print(record.features[i])


**Q3: Os resultados desta última consulta são consistentes com o que você esperaria? Por que?**

Além destas características, talvez a característica mais importante seja a **sequência** que buscamos. Obviamente, também queremos acessar esta informação através da variável *record* que criamos anteriormente.

In [ ]:
my_seq = record.seq
print(my_seq)

my_seq_length = len(my_seq)
gc_content = float(my_seq.count("G") + my_seq.count("C")) / len(my_seq)

print("The received sequence %s has %d bp and a GC fraction of %f" % (record.id, my_seq_length, gc_content))


Vamos analisar o produto da tradução desta região do genoma do vírus SARS-Cov2 e ver o que podemos descobrir sobre este fragmento do genoma.

Para isto, vamos usar a ferramenta de tradução do próprio BioPython.

In [ ]:
my_seq_trad = my_seq.translate()

print (my_seq_trad)


Na terceira etapa desta prática vamos estender a nossa análise desta sequência realizando uma busca com a ferramenta BLAST.

# Busca de Sequências com o BLAST #

Nesta última etapa da prática usaremos a ferramenta BLAST para consultar a base de dados **REFSEQ** do NCBI por homólogos da proteína que acabamos de traduzir a partir do genoma do vírus SARS-Cov2.

Nas linhas de código abaixo, usamos a consulta (online) com o BLAST a partir do BioPython. Para a consulta, precisamos dar 3 argumentos para o programa:

1.   Qual a versão do programa vamos usar: **blastp** (proteínas), **blastx** (nucleotídeo vs proteína), **tblastn** (proteína vs nucleotídeos), **tblastx** (nucleotídeo vs nucleotídeo).
2.   Qual a base de dados que consultaremos (*nr, refseq_rna, refseq_representative_genomes, refseq_protein, pdb, etc*).
3. A sequência que buscaremos (sequência ou código de acesso NCBI).



In [18]:

result_handle = NCBIWWW.qblast("blastp", "refseq_protein", my_seq_trad)


Vamos escrever os resultados em um arquivo **my_blast.xml**. Em seguida usaremos uma ferramenta do Biopython para a leitura dos dados deste arquivo XML.


In [19]:
with open("my_blast.xml", "w") as out_handle:
  out_handle.write(result_handle.read())
result_handle.close()

result_handle = open("my_blast.xml")
from Bio.Blast import NCBIXML
blast_record = NCBIXML.read(result_handle)

Com este parser dos resultados em XML, mantemos os resultados na variável blast_record. Nela temos acesso às informações que obtivemos da nossa consulta na base de dados, com por exemplo:



*   O título da sequência;
*   O tamanho da sequência (número de aminoácidos, neste caso);
* O valor de *e-value* (significância do alinhamento)
* O próprio alinhamento entre a sequência de busca e a sequência obtida da base de dados.

Para facilitar a visualização, aplicamos um critério de corte de e-value de $1 \times 10^-10$. Isto é, somente serão mostrados as informações para as sequências que têm e-value menor que este parâmetro de corte. 

Lembrando, um e-value de $1 \times 10^-2$ significa que a chance de eu obter este alinhamento ao acaso em uma base de dados do tamanho da base usada é de 1%.



In [ ]:
E_VALUE_THRESH = 1E-10

for alignment in blast_record.alignments:
  for hsp in alignment.hsps:
    if hsp.expect < E_VALUE_THRESH:
      print()
      print("****Alignment****")
      print("sequence:", alignment.title)
      print("length:", alignment.length)
      print("e value:", hsp.expect)
      print(hsp.query[0:75] + "...")
      print(hsp.match[0:75] + "...")
      print(hsp.sbjct[0:75] + "...")
      print()

**Q4: O que podemos concluir sobre a sequência do fragmento do genoma que pesquisamos? A que proteína viral se refere? Ela é exclusiva de SARS-Cov2? Se não, em que outros organismos ela ocorre?**